In [1]:
import json
import numpy as np
import pandas as pd
from typing import Dict, List, Any

class TrafficDataProcessor:
    def __init__(self, q_table_path: str):
        """Initialize with path to Q-table JSON file"""
        self.q_table_path = q_table_path
        self.q_table = self.load_q_table()
        
    def load_q_table(self) -> Dict:
        """Load and preprocess Q-table from JSON"""
        with open(self.q_table_path, 'r') as f:
            raw_table = json.load(f)
        return {eval(k): v for k, v in raw_table.items()}
    
    def get_dashboard_data(self) -> Dict[str, Any]:
        """Generate all data needed for dashboard"""
        return {
            'action_distribution': self.calculate_action_distribution(),
            'lane_metrics': self.analyze_lane_preferences(),
            'training_progress': self.generate_training_metrics(),
            'value_stats': self.analyze_q_values()
        }
    
    def calculate_action_distribution(self) -> List[Dict]:
        """Analyze preferred actions across all states"""
        actions = []
        for values in self.q_table.values():
            actions.append(np.argmax(values))
        
        counts = pd.Series(actions).value_counts()
        total = len(actions)
        
        return [
            {'action': 'Left', 'percentage': float(counts.get(0, 0)/total * 100)},
            {'action': 'Stay', 'percentage': float(counts.get(1, 0)/total * 100)},
            {'action': 'Right', 'percentage': float(counts.get(2, 0)/total * 100)}
        ]
    
    def analyze_lane_preferences(self) -> List[Dict]:
        """Analyze lane utilization and performance"""
        lane_stats = {i: {'count': 0, 'avg_value': 0} for i in range(1, 6)}
        
        for state, values in self.q_table.items():
            lane = state[1]
            if lane in lane_stats:
                lane_stats[lane]['count'] += 1
                lane_stats[lane]['avg_value'] += np.max(values)
        
        # Calculate averages and prepare return format
        return [
            {
                'lane': lane,
                'visits': stats['count'],
                'avg_value': stats['avg_value']/stats['count'] if stats['count'] > 0 else 0,
            }
            for lane, stats in lane_stats.items()
        ]
    
    def generate_training_metrics(self) -> Dict:
        """Generate metrics showing training progress"""
        states = list(self.q_table.keys())
        time_steps = len(states)
        
        # Generate episode rewards (estimated from Q-values)
        rewards = []
        rolling_rewards = []
        window = 50  # Rolling average window
        
        for i in range(time_steps):
            if i < len(states):
                reward = np.max(self.q_table[states[i]])
                rewards.append(reward)
                
                # Calculate rolling average
                if i >= window:
                    avg = np.mean(rewards[i-window:i])
                else:
                    avg = np.mean(rewards[:i+1])
                rolling_rewards.append(avg)
        
        return {
            'episodes': list(range(len(rewards))),
            'rewards': rewards,
            'rolling_avg': rolling_rewards
        }
    
    def analyze_q_values(self) -> Dict:
        """Statistical analysis of Q-values"""
        all_values = []
        for values in self.q_table.values():
            all_values.extend(values)
            
        return {
            'mean': float(np.mean(all_values)),
            'std': float(np.std(all_values)),
            'min': float(np.min(all_values)),
            'max': float(np.max(all_values)),
            'distribution': np.histogram(all_values, bins=20)[0].tolist()
        }



In [ ]:
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import pandas as pd
# import numpy as np

# class TrafficDashboard:
#     def __init__(self, dashboard_data: Dict):
#         """Initialize dashboard with processed data"""
#         self.data = dashboard_data
        
#     def create_dashboard(self) -> go.Figure:
#         """Create complete dashboard with all visualizations"""
#         # Create figure with secondary y-axis
#         fig = make_subplots(
#             rows=2, cols=2,
#             subplot_titles=(
#                 'Training Progress',
#                 'Action Distribution',
#                 'Lane Utilization',
#                 'Q-Value Distribution'
#             )
#         )
        
#         # Add training progress plot
#         self._add_training_progress(fig, 1, 1)
        
#         # Add action distribution plot
#         self._add_action_distribution(fig, 1, 2)
        
#         # Add lane utilization plot
#         self._add_lane_metrics(fig, 2, 1)
        
#         # Add Q-value distribution plot
#         self._add_value_distribution(fig, 2, 2)
        
#         # Update layout
#         fig.update_layout(
#             height=800,
#             showlegend=True,
#             title_text="Traffic Simulation Q-Learning Analysis",
#             template='plotly_white'
#         )
        
#         return fig
    
#     def _add_training_progress(self, fig: go.Figure, row: int, col: int):
#         """Add training progress subplot"""
#         metrics = self.data['training_progress']
        
#         fig.add_trace(
#             go.Scatter(
#                 x=metrics['episodes'],
#                 y=metrics['rewards'],
#                 mode='markers',
#                 name='Episode Rewards',
#                 marker=dict(size=2, opacity=0.5)
#             ),
#             row=row, col=col
#         )
        
#         fig.add_trace(
#             go.Scatter(
#                 x=metrics['episodes'],
#                 y=metrics['rolling_avg'],
#                 mode='lines',
#                 name='Rolling Average',
#                 line=dict(color='red', width=2)
#             ),
#             row=row, col=col
#         )
        
#     def _add_action_distribution(self, fig: go.Figure, row: int, col: int):
#         """Add action distribution subplot"""
#         actions = self.data['action_distribution']
        
#         fig.add_trace(
#             go.Bar(
#                 x=[d['action'] for d in actions],
#                 y=[d['percentage'] for d in actions],
#                 name='Action Distribution'
#             ),
#             row=row, col=col
#         )
        
#     def _add_lane_metrics(self, fig: go.Figure, row: int, col: int):
#         """Add lane utilization subplot"""
#         lanes = self.data['lane_metrics']
        
#         fig.add_trace(
#             go.Bar(
#                 x=[d['lane'] for d in lanes],
#                 y=[d['visits'] for d in lanes],
#                 name='Lane Visits'
#             ),
#             row=row, col=col
#         )
        
#         # Add average value line on secondary y-axis
#         fig.add_trace(
#             go.Scatter(
#                 x=[d['lane'] for d in lanes],
#                 y=[d['avg_value'] for d in lanes],
#                 name='Avg Q-Value',
#                 mode='lines+markers',
#                 line=dict(color='red')
#             ),
#             row=row, col=col
#         )
        
#     def _add_value_distribution(self, fig: go.Figure, row: int, col: int):
#         """Add Q-value distribution subplot"""
#         values = self.data['value_stats']
        
#         fig.add_trace(
#             go.Histogram(
#                 x=values['distribution'],
#                 name='Q-Value Distribution',
#                 nbinsx=20
#             ),
#             row=row, col=col
#         )

# # # Example usage
# # if __name__ == "__main__":
# #     # Load processed data
# #     with open('processed_dashboard_data.json', 'r') as f:
# #         dashboard_data = json.load(f)
    
# #     # Create and show dashboard
# #     dashboard = TrafficDashboard(dashboard_data)
# #     fig = dashboard.create_dashboard()
# #     fig.show()

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

class TrafficDashboard:
    def __init__(self, dashboard_data: Dict):
        self.data = dashboard_data
        
    def create_dashboard(self) -> go.Figure:
        # Create figure with subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Training Progress',
                'Action Distribution',
                'Lane Utilization',
                'Q-Value Distribution'
            )
        )
        
        # Add all subplots
        self._add_training_progress(fig, 1, 1)
        self._add_action_distribution(fig, 1, 2)
        self._add_lane_metrics(fig, 2, 1)
        self._add_value_distribution(fig, 2, 2)
        
        # Update overall layout
        fig.update_layout(
            height=800,
            showlegend=True,
            title_text="Traffic Simulation Q-Learning Analysis",
            template='plotly_white',
            font=dict(size=12)
        )
        
        return fig
    
    def _add_training_progress(self, fig: go.Figure, row: int, col: int):
        """Add training progress subplot with improved axes"""
        metrics = self.data['training_progress']
        
        # Add episode rewards scatter
        fig.add_trace(
            go.Scatter(
                x=metrics['episodes'],
                y=metrics['rewards'],
                mode='markers',
                name='Episode Rewards',
                marker=dict(
                    size=4,
                    opacity=0.5,
                    color='blue'
                ),
                hovertemplate="Episode: %{x}<br>Reward: %{y:.2f}<extra></extra>"
            ),
            row=row, col=col
        )
        
        # Add rolling average line
        fig.add_trace(
            go.Scatter(
                x=metrics['episodes'],
                y=metrics['rolling_avg'],
                mode='lines',
                name='Rolling Average (50 episodes)',
                line=dict(
                    color='red',
                    width=2
                ),
                hovertemplate="Episode: %{x}<br>Avg Reward: %{y:.2f}<extra></extra>"
            ),
            row=row, col=col
        )
        
        # Update axes for this subplot
        fig.update_xaxes(
            title_text="Training Episodes",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            gridcolor='lightgray',
            row=row, col=col
        )
        
        fig.update_yaxes(
            title_text="Cumulative Reward",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            gridcolor='lightgray',
            row=row, col=col
        )
        
        # Add axis annotations
        fig.add_annotation(
            text="Episode rewards include:<br>• Distance covered<br>• Lane change penalties (-5)<br>• Time penalties (-10)",
            xref="paper", yref="paper",
            x=0.02, y=0.98,
            showarrow=False,
            font=dict(size=10),
            align="left",
            bgcolor="rgba(255,255,255,0.8)",
            row=row, col=col
        )
    
    def _add_action_distribution(self, fig: go.Figure, row: int, col: int):
        """Add action distribution subplot with improved axes"""
        actions = self.data['action_distribution']
        
        fig.add_trace(
            go.Bar(
                x=[d['action'] for d in actions],
                y=[d['percentage'] for d in actions],
                name='Action Distribution',
                text=[f"{d['percentage']:.1f}%" for d in actions],
                textposition='auto',
            ),
            row=row, col=col
        )
        
        fig.update_xaxes(
            title_text="Agent Actions",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
        
        fig.update_yaxes(
            title_text="Selection Frequency (%)",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            range=[0, 100],
            row=row, col=col
        )
    
    def _add_lane_metrics(self, fig: go.Figure, row: int, col: int):
        """Add lane utilization subplot with improved axes"""
        lanes = self.data['lane_metrics']
        
        # Bar chart for visits
        fig.add_trace(
            go.Bar(
                x=[d['lane'] for d in lanes],
                y=[d['visits'] for d in lanes],
                name='Lane Visits',
                text=[f"{d['visits']}" for d in lanes],
                textposition='auto',
            ),
            row=row, col=col
        )
        
        # Line for average Q-values
        fig.add_trace(
            go.Scatter(
                x=[d['lane'] for d in lanes],
                y=[d['avg_value'] for d in lanes],
                name='Avg Q-Value',
                mode='lines+markers',
                line=dict(color='red'),
                yaxis='y2'
            ),
            row=row, col=col
        )
        
        fig.update_xaxes(
            title_text="Lane Number",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
        
        fig.update_yaxes(
            title_text="Visit Count",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
        
        # Add secondary y-axis for Q-values
        fig.update_yaxes(
            title_text="Average Q-Value",
            secondary_y=True,
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
    
    def _add_value_distribution(self, fig: go.Figure, row: int, col: int):
        """Add Q-value distribution subplot with improved axes"""
        values = self.data['value_stats']
        
        fig.add_trace(
            go.Histogram(
                x=values['distribution'],
                name='Q-Value Distribution',
                nbinsx=20,
                histnorm='percent'
            ),
            row=row, col=col
        )
        
        fig.update_xaxes(
            title_text="Q-Value Range",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
        
        fig.update_yaxes(
            title_text="Frequency (%)",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )


In [4]:
# Example usage
processor = TrafficDataProcessor('Q_learning_best_model.json')
dashboard_data = processor.get_dashboard_data()

In [ ]:
dashboard = TrafficDashboard(dashboard_data)
fig = dashboard.create_dashboard()
fig.show()

In [7]:
dashboard = TrafficDashboard(dashboard_data)
fig = dashboard.create_dashboard()

# Update figure layout for better visibility
fig.update_layout(
    title=dict(
        text="Traffic Simulation Q-Learning Analysis",
        x=0.5,
        y=0.95,
        xanchor='center',
        font=dict(size=20)
    ),
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255,255,255,0.8)"
    ),
    margin=dict(t=100, b=50, l=50, r=50))

fig.show()